# Geocode Singapore Addresses

In [1]:
import pandas as pd
import glob
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
resale_price_all = glob.glob("./*.csv")

li = []

for filename in resale_price_all:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

resale_df = pd.concat(li, axis=0, ignore_index=True)
resale_df.head()
resale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746207 entries, 0 to 746206
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                746207 non-null  object 
 1   town                 746207 non-null  object 
 2   flat_type            746207 non-null  object 
 3   block                746207 non-null  object 
 4   street_name          746207 non-null  object 
 5   storey_range         746207 non-null  object 
 6   floor_area_sqm       746207 non-null  float64
 7   flat_model           746207 non-null  object 
 8   lease_commence_date  746207 non-null  int64  
 9   resale_price         746207 non-null  float64
 10  remaining_lease      37153 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 62.6+ MB


In [3]:
resale_df['ADDRESS'] = df['block'].astype(str) + ' ' + df['street_name'] + ',' + ' Singapore'   

resale_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,"174 ANG MO KIO AVE 4, Singapore"
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN,"541 ANG MO KIO AVE 10, Singapore"
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN,"163 ANG MO KIO AVE 4, Singapore"
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN,"446 ANG MO KIO AVE 10, Singapore"
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN,"557 ANG MO KIO AVE 10, Singapore"


In [4]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.2)

In [5]:
resale_df['location'] = resale_df['ADDRESS'].apply(geocode)
resale_df.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl

RateLimiter swallowed an error after 2 retries. Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl.py

RateLimiter caught an error, retrying (1/2 tries). Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl

RateLimiter caught an error, retrying (0/2 tries). Called with (*('852 HOUGANG CTRL, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "/Users/K/miniconda/lib/pyth

RateLimiter caught an error, retrying (1/2 tries). Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl

...

RateLimiter caught an error, retrying (1/2 tries). Called with (*('140 BEDOK NTH ST 2, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/Users/K/miniconda/lib/python3.8/site-p

RateLimiter caught an error, retrying (0/2 tries). Called with (*('211 BT BATOK ST 21, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
OSError: [Errno 50] Network is down

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "/Users/K/minico

RateLimiter caught an error, retrying (0/2 tries). Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl

RateLimiter swallowed an error after 2 retries. Called with (*('2 HOLLAND AVE, Singapore',), **{}).
Traceback (most recent call last):
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/K/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/K/miniconda/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/K/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/Users/K/miniconda/lib/python3.8/ssl.py

KeyboardInterrupt: 

In [ ]:
resale_df['point'] = resale_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
resale_df[['latitude', 'longitude','altitude']] = pd.DataFrame(resale_df['point'].tolist(), index=resale_df.index)
resale_df.head()

In [ ]:
resale_df = resale_df.drop(['block', 'street_name', 'point', 'location', 'altitude'], axis=1)
resale_df.head()

In [11]:
resale_df.to_csv('./resale.csv', index=False)